In [154]:
# Controle data .gouv
# 1. caracteristics
import pandas as pd
import numpy as np

# Chemins d'accès aux données
repData = '../../dataBrutes.gouv/'
repDataConcat = '../../dataConcat.gouv/'

# Liste des fichiers usagers disponibles
!ls {repData}us* -lrt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

ls: -lrt: No such file or directory
../../dataBrutes.gouv/usagers-2017.csv ../../dataBrutes.gouv/usagers_2008.csv
../../dataBrutes.gouv/usagers-2018.csv ../../dataBrutes.gouv/usagers_2009.csv
../../dataBrutes.gouv/usagers-2019.csv ../../dataBrutes.gouv/usagers_2010.csv
../../dataBrutes.gouv/usagers-2020.csv ../../dataBrutes.gouv/usagers_2011.csv
../../dataBrutes.gouv/usagers-2021.csv ../../dataBrutes.gouv/usagers_2012.csv
../../dataBrutes.gouv/usagers-2022.csv ../../dataBrutes.gouv/usagers_2013.csv
../../dataBrutes.gouv/usagers_2005.csv ../../dataBrutes.gouv/usagers_2014.csv
../../dataBrutes.gouv/usagers_2006.csv ../../dataBrutes.gouv/usagers_2015.csv
../../dataBrutes.gouv/usagers_2007.csv ../../dataBrutes.gouv/usagers_2016.csv


#### Consolidation de 'usagers.csv' - modus operandi
0. Chgt de tous les fichiers 
      - nom des fichiers de 2005 à 2016 : '_'
      - nom des fichiers de 2017 à 2022 : remplacement de '_' par '-'
      - séparateur ',' sauf 2009='\t' et 2019 à 2022=';'
1. Variable 'Num_Acc' renommée en 'id_accident' pour être iso entre les 4 types de fichiers
2. Variable 'id_vehicule' : apparition en 2019.
      - Création d'une colonne 'id_vehicule' dans tous les fichiers antérieurs à 2019 initialisée à 0 afin de l'utiliser dans les jointures
      - Force le type de la variable en int64 (remplacement de 'xa0' par '' pour cela
        
3. Variable 'id_usager' : apparition en 2021 (utilisation ?)
      - Création d'une colonne 'id_usager' dans tous les fichiers antérieurs à 2021 au cas où l'info soit exploitable

4. Variable 'place' : Remplacement dans tous les fichiers :
      - Valeur NaN  --> -1=non renseigné (apparue en 2019)
      - forcer le type en int64
5. Variable 'grav' : La catégorisation des blessés a changé en 2018 où la différenciation Blessé léger et grave n'est plus faite
      - Regroupement des 4 - Blessé léger en 3=Blessé sur tous les fichiers antérieurs à 2018 en première intention
    
    /!\ /!\ Finalement annulation de ce regroupement /!\ /!\

   Décision de transcoder les valeurs de cette variable afin qu'elles soient ordonnées :
   grav                  ----> gravite                 Nous profiterons de cette modification pour modifier le nom de la variable
   1= indemne            ----> 0 = indemne
   4= blessé léger       ----> 1 = Blessé léger
   3= blessé hospitalisé ----> 2 = Blessé hospitalisé
   2= tué                ----> 3 = tué
 
6. Variable 'trajet' : Il existe une catégorie 0: non renseigné ainsi que -1
      - passer les NaN en -1 
      - passer tous les 0 en -1
7. Variable Sécu(1) / secu2 / secu3:
      - avant 2019 : un seul équipement géré : codé sur 2 caractère (secu=car1+car2)
    le premier concerne l’existence d’un Équipement de sécurité --> car1
         1 – Ceinture
         2 – Casque
         3 – Dispositif enfants
         4 – Equipement réfléchissant
         9 – Autre

    le second concerne l’utilisation de l’Équipement de sécurité --> car2
         1 – Oui
         2 – Non
         3 – Non déterminable

    --> Transcoder ces données de manière à renseigner secu1 comme suit :
        Si car2 == 2 --> secu1=0
        Si car2 == 3 --> secu1=8
        Si car2 == 1 --> secu1=car1
      - Variables 'secu2' et 'secu3' : avant 2019, un seul équipement était géré, créer secu2 et secu3 --> -1
8. Variable 'locp' : Beaucoup de valeurs -1=non renseigné sur des enregistrements d'usager non piéton 
      - Passer les fichiers antérieurs à 2019 en NaN --> -1
      - Forcer à 0=Sans objet tous les enregistrements à -1 ne concernant pas les piéton (catu!=3)
      - Typer la variable en 'int64'
  
      - Question : il existe une valeur 9=inconnue ==> ne faudrait-il pas forcer les -1 (pour les usager=piéton) à cette valeur ?
    ==> décision de laisser en l'état, les choix se prendront ultérieurement selon les contraintes rencontrées.
9. Variable 'actp' : Beaucoup de valeurs -1=non renseigné sur des enregistrements d'usager non piéton
      - A partir de 2019, des valeurs en alpha apparaissent : soit on les garde et il faut passer la colonne en str dans tous les df
    soit on transcode en numérique ces valeurs alpha
    Décision de laisser les données en l'état.
    --> Pour permettre le typage en str, il faut pour chaque df antérieures à 2019 caster en premier lieu en int64 puis en str.
    (nécessaire pour que in fine nous n'ayons pas du 3 et du 3.0 qui se retrouve dans la variable.
    
      - Passer les fichiers antérieurs à 2019 en NaN --> -1
      - Forcer à 0=Sans objet tous les enregistrements à -1 ne concernant pas les piéton (catu!=3)
      - Dans les postérieurs à 2018, le -1 possède un espace devant qu'il faut supprimer.
10. Variable 'etatp' : Beaucoup de valeurs -1=non renseigné sur des enregistrements d'usager non piéton 
      - Passer les fichiers antérieurs à 2019 en NaN --> -1
      - Forcer à 0=Sans objet tous les enregistrements à -1 ne concernant pas les piéton (catu!=3)
      - Typer la variable en 'int64' (car des codes alpha apparaissent à partir de 2019.

      - A partir de 2019 : nouvelles valeurs (alpha) : Quid de l'historique des données ? Autre 

11. Ordonner les colonnes de chaque df dans le mêm1 ordre
12. Concaténation en un seul f2chier 
13. Variable 'catv' année 2005, remplacement des valeurs qui n'ont pu l'être précédemment
14. Libération de mémoire  
15. Variable 'catu' : Une modalité disparait à partir de 2018 de cette variable (4=Piéton en Roller ou trotinette) pour être reventilée
    dans véhicule.catv (valeur 99).
    Maintien de cette valeur 4 qu'il faudrait déplacer dans véhicule (à analyser ultérieurement)
16. Suppression des colonnes inutiles
17. Sauvegarde du df consolidé sous un fichier caracteristiques.csv

In [155]:
# 0. chargement et merge de tous les fichiers de 2005 à 2016 --> carac
df_2005 = pd.read_csv(repData + "usagers_2005.csv", sep = ',')
df_2006 = pd.read_csv(repData + "usagers_2006.csv", sep = ',')
df_2007 = pd.read_csv(repData + "usagers_2007.csv", sep = ',')
df_2008 = pd.read_csv(repData + "usagers_2008.csv", sep = ',')
df_2009 = pd.read_csv(repData + "usagers_2009.csv", sep = ',')
df_2010 = pd.read_csv(repData + "usagers_2010.csv", sep = ',')
df_2011 = pd.read_csv(repData + "usagers_2011.csv", sep = ',')
df_2012 = pd.read_csv(repData + "usagers_2012.csv", sep = ',')
df_2013 = pd.read_csv(repData + "usagers_2013.csv", sep = ',')
df_2014 = pd.read_csv(repData + "usagers_2014.csv", sep = ',')
df_2015 = pd.read_csv(repData + "usagers_2015.csv", sep = ',')
df_2016 = pd.read_csv(repData + "usagers_2016.csv", sep = ',')
df_2017 = pd.read_csv(repData + "usagers-2017.csv", sep = ',')
df_2018 = pd.read_csv(repData + "usagers-2018.csv", sep = ',')
df_2019 = pd.read_csv(repData + "usagers-2019.csv", sep = ';')
df_2020 = pd.read_csv(repData + "usagers-2020.csv", sep = ';')
df_2021 = pd.read_csv(repData + "usagers-2021.csv", sep = ';')
df_2022 = pd.read_csv(repData + "usagers-2022.csv", sep = ';')


In [156]:
df_2018.grav.value_counts()

grav
1    54248
4    50360
3    22169
2     3392
Name: count, dtype: int64

In [157]:
# 1. Renommer la colonne  'Num_Acc' en 'id_accident' qui servira de clé entre les 4 types de fichiers
rename_columns = {'Num_Acc' : 'id_accident'}
df_2005 = df_2005.rename(columns = rename_columns)
df_2006 = df_2006.rename(columns = rename_columns)
df_2007 = df_2007.rename(columns = rename_columns)
df_2008 = df_2008.rename(columns = rename_columns)
df_2009 = df_2009.rename(columns = rename_columns)
df_2010 = df_2010.rename(columns = rename_columns)
df_2011 = df_2011.rename(columns = rename_columns)
df_2012 = df_2012.rename(columns = rename_columns)
df_2013 = df_2013.rename(columns = rename_columns)
df_2014 = df_2014.rename(columns = rename_columns)
df_2015 = df_2015.rename(columns = rename_columns)
df_2016 = df_2016.rename(columns = rename_columns)
df_2017 = df_2017.rename(columns = rename_columns)
df_2018 = df_2018.rename(columns = rename_columns)
df_2019 = df_2019.rename(columns = rename_columns)
df_2020 = df_2020.rename(columns = rename_columns)
df_2021 = df_2021.rename(columns = rename_columns)
df_2022 = df_2022.rename(columns = rename_columns)


In [158]:
# 2.1 Variable 'id_vehicule' ajouté dans tous les fichiers antérieurs à 2019 
df_2005['id_vehicule'] = 0
df_2006['id_vehicule'] = 0
df_2007['id_vehicule'] = 0
df_2008['id_vehicule'] = 0
df_2009['id_vehicule'] = 0
df_2010['id_vehicule'] = 0
df_2011['id_vehicule'] = 0
df_2012['id_vehicule'] = 0
df_2013['id_vehicule'] = 0
df_2014['id_vehicule'] = 0
df_2015['id_vehicule'] = 0
df_2016['id_vehicule'] = 0
df_2017['id_vehicule'] = 0
df_2018['id_vehicule'] = 0

# 2.2 Variable 'id_vehicule' : retirer le séparateur de millier ' ' --> transtyper la variable en 'int64'
df_2005['id_vehicule'] = df_2005['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2006['id_vehicule'] = df_2006['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2007['id_vehicule'] = df_2007['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2008['id_vehicule'] = df_2008['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2009['id_vehicule'] = df_2009['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2010['id_vehicule'] = df_2010['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2011['id_vehicule'] = df_2011['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2012['id_vehicule'] = df_2012['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2013['id_vehicule'] = df_2013['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2014['id_vehicule'] = df_2014['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2015['id_vehicule'] = df_2015['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2016['id_vehicule'] = df_2016['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2017['id_vehicule'] = df_2017['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2018['id_vehicule'] = df_2018['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2019['id_vehicule'] = df_2019['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2020['id_vehicule'] = df_2020['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2021['id_vehicule'] = df_2021['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)
df_2022['id_vehicule'] = df_2022['id_vehicule'].astype(str).str.replace('\xa0', '').astype(int)


In [159]:
# 3. Variable 'id_usager' ajouté dans tous les fichiers antérieurs à 2021 : le ctrl de doublon ne sera possible qu'à partir de 2021
df_2005['id_usager'] = np.nan
df_2006['id_usager'] = np.nan
df_2007['id_usager'] = np.nan
df_2008['id_usager'] = np.nan
df_2009['id_usager'] = np.nan
df_2010['id_usager'] = np.nan
df_2011['id_usager'] = np.nan
df_2012['id_usager'] = np.nan
df_2013['id_usager'] = np.nan
df_2014['id_usager'] = np.nan
df_2015['id_usager'] = np.nan
df_2016['id_usager'] = np.nan
df_2017['id_usager'] = np.nan
df_2018['id_usager'] = np.nan
df_2019['id_usager'] = np.nan
df_2020['id_usager'] = np.nan


In [160]:
# 4. Variable 'place' : passer les NaN -->  '-1'  et forcer le type en 'int64'
df_2005["place"] = df_2005["place"].fillna(-1).astype(int)
df_2006["place"] = df_2006["place"].fillna(-1).astype(int)
df_2007["place"] = df_2007["place"].fillna(-1).astype(int)
df_2008["place"] = df_2008["place"].fillna(-1).astype(int)
df_2009["place"] = df_2009["place"].fillna(-1).astype(int)
df_2010["place"] = df_2010["place"].fillna(-1).astype(int)
df_2011["place"] = df_2011["place"].fillna(-1).astype(int)
df_2012["place"] = df_2012["place"].fillna(-1).astype(int)
df_2013["place"] = df_2013["place"].fillna(-1).astype(int)
df_2014["place"] = df_2014["place"].fillna(-1).astype(int)
df_2015["place"] = df_2015["place"].fillna(-1).astype(int)
df_2016["place"] = df_2016["place"].fillna(-1).astype(int)
df_2017["place"] = df_2017["place"].fillna(-1).astype(int)
df_2018["place"] = df_2018["place"].fillna(-1).astype(int)
df_2019["place"] = df_2019["place"].fillna(-1).astype(int)
df_2020["place"] = df_2020["place"].fillna(-1).astype(int)
df_2021["place"] = df_2021["place"].fillna(-1).astype(int)
df_2022["place"] = df_2022["place"].fillna(-1).astype(int)


In [161]:
# 5.0 Variable 'grav' : force les valeurs 4 --> 3 : regroupement des blessés légers et grave en une seule catégorie
if 1 == 0 :
    # Finalement ce regroupement n'est pas réalisé dès lors que l'on se limite à l'usage des données > 2018 autant garder les 4 modalités.
    df_2005["grav"] = df_2005["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2006["grav"] = df_2006["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2007["grav"] = df_2007["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2008["grav"] = df_2008["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2009["grav"] = df_2009["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2010["grav"] = df_2010["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2011["grav"] = df_2011["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2012["grav"] = df_2012["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2013["grav"] = df_2013["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2014["grav"] = df_2014["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2015["grav"] = df_2015["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2016["grav"] = df_2016["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2017["grav"] = df_2017["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2018["grav"] = df_2018["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2019["grav"] = df_2019["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2020["grav"] = df_2020["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2021["grav"] = df_2021["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)
    df_2022["grav"] = df_2022["grav"].apply (lambda valeur : 3 if valeur == 4 else valeur)

# Ordonne les modalités de gravité et repart de 0..3
df_2005["gravite"] = df_2005["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2006["gravite"] = df_2006["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2007["gravite"] = df_2007["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2008["gravite"] = df_2008["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2009["gravite"] = df_2009["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2010["gravite"] = df_2010["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2011["gravite"] = df_2011["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2012["gravite"] = df_2012["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2013["gravite"] = df_2013["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2014["gravite"] = df_2014["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2015["gravite"] = df_2015["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2016["gravite"] = df_2016["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2017["gravite"] = df_2017["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2018["gravite"] = df_2018["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2019["gravite"] = df_2019["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2020["gravite"] = df_2020["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2021["gravite"] = df_2021["grav"].replace({1:0, 2:3, 3:2, 4: 1})
df_2022["gravite"] = df_2022["grav"].replace({1:0, 2:3, 3:2, 4: 1})


In [162]:
# 6. Variable 'trajet' : NaN et 0=non renseigné --> -1=Non renseigné + typage en int
df_2005["trajet"] = df_2005["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2006["trajet"] = df_2006["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2007["trajet"] = df_2007["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2008["trajet"] = df_2008["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2009["trajet"] = df_2009["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2010["trajet"] = df_2010["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2011["trajet"] = df_2011["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2012["trajet"] = df_2012["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2013["trajet"] = df_2013["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2014["trajet"] = df_2014["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2015["trajet"] = df_2015["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2016["trajet"] = df_2016["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2017["trajet"] = df_2017["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2018["trajet"] = df_2018["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2019["trajet"] = df_2019["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2020["trajet"] = df_2020["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2021["trajet"] = df_2021["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)
df_2022["trajet"] = df_2022["trajet"].fillna(-1).apply(lambda valeur : -1 if valeur == 0 else valeur).astype(int)



In [163]:
# 7.1 Fonction de transcodage de secu --> secu1
def transcode_secu(valeur:int)->int:
    """ Fonction a utiliser dans un apply()
    Secu est codé sur 2 caractères :
     le premier concerne l’existence d’un Équipement de sécurité --> car1
     1 – Ceinture
     2 – Casque
     3 – Dispositif enfants
     4 – Equipement réfléchissant
     9 – Autre

     le second concerne l’utilisation de l’Équipement de sécurité --> car2
     1 – Oui
     2 – Non
     3 – Non déterminable

    --> Transcoder ces données de manière à renseigner secu1 comme suit :
    Si car2 == 2 --> secu1=0
    Si car2 == 3 --> secu1=8
    Si car2 == 1 --> secu1=car1
    Sinon secu1 --> 0
    
    Args :
    - int  : Dataframe à traiter. Il ne sera pas modifié par la fonction

    Return :
    - int transcodé
    
    """
    try:
        valeur = int(valeur)
        
    except :
        # La valeur est Nan --> -1
        return -1
    else :
        
        try :
            car1 = str(valeur)[0:1]
            car2 = str(valeur)[1:]
            if car2 == '2' :
                return 0
            elif car2 == '3' :
                return 8
            elif car2 == '1' :
                return int(car1)
        except :
            None
        
    return 0

    


In [164]:
# 7.2 Variable 'secu1' = f(secu) pour les fichiers antérieurs à 2019
df_2005['secu1'] = df_2005['secu'].apply(transcode_secu).astype(int)
df_2006['secu1'] = df_2006['secu'].apply(transcode_secu).astype(int)
df_2007['secu1'] = df_2007['secu'].apply(transcode_secu).astype(int)
df_2008['secu1'] = df_2008['secu'].apply(transcode_secu).astype(int)
df_2009['secu1'] = df_2009['secu'].apply(transcode_secu).astype(int)
df_2010['secu1'] = df_2010['secu'].apply(transcode_secu).astype(int)
df_2011['secu1'] = df_2011['secu'].apply(transcode_secu).astype(int)
df_2012['secu1'] = df_2012['secu'].apply(transcode_secu).astype(int)
df_2013['secu1'] = df_2013['secu'].apply(transcode_secu).astype(int)
df_2014['secu1'] = df_2014['secu'].apply(transcode_secu).astype(int)
df_2015['secu1'] = df_2015['secu'].apply(transcode_secu).astype(int)
df_2016['secu1'] = df_2016['secu'].apply(transcode_secu).astype(int)
df_2017['secu1'] = df_2017['secu'].apply(transcode_secu).astype(int)
df_2018['secu1'] = df_2018['secu'].apply(transcode_secu).astype(int)


In [165]:
# 7.3 Variables 'secu2' et 'secu3' à créer à -1 et typage en 'int64' pour les fichiers antérieurs à 2019
df_2005['secu2'] = -1
df_2005['secu3'] = -1
df_2006['secu2'] = -1
df_2006['secu3'] = -1
df_2007['secu2'] = -1
df_2007['secu3'] = -1
df_2008['secu2'] = -1
df_2008['secu3'] = -1
df_2009['secu2'] = -1
df_2009['secu3'] = -1
df_2010['secu2'] = -1
df_2010['secu3'] = -1
df_2011['secu2'] = -1
df_2011['secu3'] = -1
df_2012['secu2'] = -1
df_2012['secu3'] = -1
df_2013['secu2'] = -1
df_2013['secu3'] = -1
df_2014['secu2'] = -1
df_2014['secu3'] = -1
df_2015['secu2'] = -1
df_2015['secu3'] = -1
df_2016['secu2'] = -1
df_2016['secu3'] = -1
df_2017['secu2'] = -1
df_2017['secu3'] = -1
df_2018['secu2'] = -1
df_2018['secu3'] = -1


In [166]:
# 8.1 Variable 'locp' contient des valeurs NaN avant 2019 alors que ces mêmes enregistrements sont à -1 à partir de 2019
df_2005['locp'] = df_2005['locp'].fillna(-1).astype(int)
df_2006['locp'] = df_2006['locp'].fillna(-1).astype(int)
df_2007['locp'] = df_2007['locp'].fillna(-1).astype(int)
df_2008['locp'] = df_2008['locp'].fillna(-1).astype(int)
df_2009['locp'] = df_2009['locp'].fillna(-1).astype(int)
df_2010['locp'] = df_2010['locp'].fillna(-1).astype(int)
df_2011['locp'] = df_2011['locp'].fillna(-1).astype(int)
df_2012['locp'] = df_2012['locp'].fillna(-1).astype(int)
df_2013['locp'] = df_2013['locp'].fillna(-1).astype(int)
df_2014['locp'] = df_2014['locp'].fillna(-1).astype(int)
df_2015['locp'] = df_2015['locp'].fillna(-1).astype(int)
df_2016['locp'] = df_2016['locp'].fillna(-1).astype(int)
df_2017['locp'] = df_2017['locp'].fillna(-1).astype(int)
df_2018['locp'] = df_2018['locp'].fillna(-1).astype(int)
# Même si inutle (plus de NaN à partir de 2019 --> maintien de l'opération au moins pour forcer le typage
df_2019['locp'] = df_2019['locp'].fillna(-1).astype(int)
df_2020['locp'] = df_2020['locp'].fillna(-1).astype(int)
df_2021['locp'] = df_2021['locp'].fillna(-1).astype(int)
df_2022['locp'] = df_2022['locp'].fillna(-1).astype(int)


In [167]:
# 8.2 Variable 'locp' ne concerne que les piéton : il manque des infos (-1) pour des non piétons --> 0=non concerné
df_2005.loc[(df_2005['catu'] != 3) & (df_2005['locp'] == -1), 'locp'] = 0
df_2006.loc[(df_2006['catu'] != 3) & (df_2006['locp'] == -1), 'locp'] = 0
df_2007.loc[(df_2007['catu'] != 3) & (df_2007['locp'] == -1), 'locp'] = 0
df_2008.loc[(df_2008['catu'] != 3) & (df_2008['locp'] == -1), 'locp'] = 0
df_2009.loc[(df_2009['catu'] != 3) & (df_2009['locp'] == -1), 'locp'] = 0
df_2010.loc[(df_2010['catu'] != 3) & (df_2010['locp'] == -1), 'locp'] = 0
df_2011.loc[(df_2011['catu'] != 3) & (df_2011['locp'] == -1), 'locp'] = 0
df_2012.loc[(df_2012['catu'] != 3) & (df_2012['locp'] == -1), 'locp'] = 0
df_2013.loc[(df_2013['catu'] != 3) & (df_2013['locp'] == -1), 'locp'] = 0
df_2014.loc[(df_2014['catu'] != 3) & (df_2014['locp'] == -1), 'locp'] = 0
df_2015.loc[(df_2015['catu'] != 3) & (df_2015['locp'] == -1), 'locp'] = 0
df_2016.loc[(df_2016['catu'] != 3) & (df_2016['locp'] == -1), 'locp'] = 0
df_2017.loc[(df_2017['catu'] != 3) & (df_2017['locp'] == -1), 'locp'] = 0
df_2018.loc[(df_2018['catu'] != 3) & (df_2018['locp'] == -1), 'locp'] = 0
df_2019.loc[(df_2019['catu'] != 3) & (df_2019['locp'] == -1), 'locp'] = 0
df_2020.loc[(df_2020['catu'] != 3) & (df_2020['locp'] == -1), 'locp'] = 0
df_2021.loc[(df_2021['catu'] != 3) & (df_2021['locp'] == -1), 'locp'] = 0
df_2022.loc[(df_2022['catu'] != 3) & (df_2022['locp'] == -1), 'locp'] = 0


In [168]:
# 9.1 Variable 'actp' contient des valeurs NaN avant 2019 alors que ces mêmes enregistrements sont à -1 à partir de 2019
# typage en str car des valeurs alph apparaissent à partir de 2019
df_2005['actp'] = df_2005['actp'].fillna(-1).astype(int).astype(str)
df_2006['actp'] = df_2006['actp'].fillna(-1).astype(int).astype(str)
df_2007['actp'] = df_2007['actp'].fillna(-1).astype(int).astype(str)
df_2008['actp'] = df_2008['actp'].fillna(-1).astype(int).astype(str)
df_2009['actp'] = df_2009['actp'].fillna(-1).astype(int).astype(str)
df_2010['actp'] = df_2010['actp'].fillna(-1).astype(int).astype(str)
df_2011['actp'] = df_2011['actp'].fillna(-1).astype(int).astype(str)
df_2012['actp'] = df_2012['actp'].fillna(-1).astype(int).astype(str)
df_2013['actp'] = df_2013['actp'].fillna(-1).astype(int).astype(str)
df_2014['actp'] = df_2014['actp'].fillna(-1).astype(int).astype(str)
df_2015['actp'] = df_2015['actp'].fillna(-1).astype(int).astype(str)
df_2016['actp'] = df_2016['actp'].fillna(-1).astype(int).astype(str)
df_2017['actp'] = df_2017['actp'].fillna(-1).astype(int).astype(str)
df_2018['actp'] = df_2018['actp'].fillna(-1).astype(int).astype(str)
df_2019['actp'] = df_2019['actp'].fillna(-1).astype(str)
df_2020['actp'] = df_2020['actp'].fillna(-1).astype(str)
df_2021['actp'] = df_2021['actp'].fillna(-1).astype(str)
df_2022['actp'] = df_2022['actp'].fillna(-1).astype(str)


In [169]:
# 9.3 Variable 'actp', remplacer la valeur ' -1' par '-1' dans tous les df
df_2005['actp'] = df_2005['actp'].str.replace (' -1', '-1')
df_2006['actp'] = df_2006['actp'].str.replace (' -1', '-1')
df_2007['actp'] = df_2007['actp'].str.replace (' -1', '-1')
df_2008['actp'] = df_2008['actp'].str.replace (' -1', '-1')
df_2009['actp'] = df_2009['actp'].str.replace (' -1', '-1')
df_2010['actp'] = df_2010['actp'].str.replace (' -1', '-1')
df_2011['actp'] = df_2011['actp'].str.replace (' -1', '-1')
df_2012['actp'] = df_2012['actp'].str.replace (' -1', '-1')
df_2013['actp'] = df_2013['actp'].str.replace (' -1', '-1')
df_2014['actp'] = df_2014['actp'].str.replace (' -1', '-1')
df_2015['actp'] = df_2015['actp'].str.replace (' -1', '-1')
df_2016['actp'] = df_2016['actp'].str.replace (' -1', '-1')
df_2017['actp'] = df_2017['actp'].str.replace (' -1', '-1')
df_2018['actp'] = df_2018['actp'].str.replace (' -1', '-1')
df_2019['actp'] = df_2019['actp'].str.replace (' -1', '-1')
df_2020['actp'] = df_2020['actp'].str.replace (' -1', '-1')
df_2021['actp'] = df_2021['actp'].str.replace (' -1', '-1')
df_2022['actp'] = df_2022['actp'].str.replace (' -1', '-1')

In [170]:
# 9.2 Variable 'actp' ne concerne que les piétons : il manque des infos (-1) pour des non piétons --> 0=non concerné
df_2005.loc[(df_2005['catu'] != 3) & (df_2005['actp'] == '-1'), 'actp'] = '0'
df_2006.loc[(df_2006['catu'] != 3) & (df_2006['actp'] == '-1'), 'actp'] = '0'
df_2007.loc[(df_2007['catu'] != 3) & (df_2007['actp'] == '-1'), 'actp'] = '0'
df_2008.loc[(df_2008['catu'] != 3) & (df_2008['actp'] == '-1'), 'actp'] = '0'
df_2009.loc[(df_2009['catu'] != 3) & (df_2009['actp'] == '-1'), 'actp'] = '0'
df_2010.loc[(df_2010['catu'] != 3) & (df_2010['actp'] == '-1'), 'actp'] = '0'
df_2011.loc[(df_2011['catu'] != 3) & (df_2011['actp'] == '-1'), 'actp'] = '0'
df_2012.loc[(df_2012['catu'] != 3) & (df_2012['actp'] == '-1'), 'actp'] = '0'
df_2013.loc[(df_2013['catu'] != 3) & (df_2013['actp'] == '-1'), 'actp'] = '0'
df_2014.loc[(df_2014['catu'] != 3) & (df_2014['actp'] == '-1'), 'actp'] = '0'
df_2015.loc[(df_2015['catu'] != 3) & (df_2015['actp'] == '-1'), 'actp'] = '0'
df_2016.loc[(df_2016['catu'] != 3) & (df_2016['actp'] == '-1'), 'actp'] = '0'
df_2017.loc[(df_2017['catu'] != 3) & (df_2017['actp'] == '-1'), 'actp'] = '0'
df_2018.loc[(df_2018['catu'] != 3) & (df_2018['actp'] == '-1'), 'actp'] = '0'
df_2019.loc[(df_2019['catu'] != 3) & (df_2019['actp'] == '-1'), 'actp'] = '0'
df_2020.loc[(df_2020['catu'] != 3) & (df_2020['actp'] == '-1'), 'actp'] = '0'
df_2021.loc[(df_2021['catu'] != 3) & (df_2021['actp'] == '-1'), 'actp'] = '0'
df_2022.loc[(df_2022['catu'] != 3) & (df_2022['actp'] == '-1'), 'actp'] = '0'


In [171]:
# 10.1 Variable 'etatp' contient des valeurs NaN avant 2019 alors que ces mêmes enregistrements sont à -1 à partir de 2019
df_2005['etatp'] = df_2005['etatp'].fillna(-1).astype(int)
df_2006['etatp'] = df_2006['etatp'].fillna(-1).astype(int)
df_2007['etatp'] = df_2007['etatp'].fillna(-1).astype(int)
df_2008['etatp'] = df_2008['etatp'].fillna(-1).astype(int)
df_2009['etatp'] = df_2009['etatp'].fillna(-1).astype(int)
df_2010['etatp'] = df_2010['etatp'].fillna(-1).astype(int)
df_2011['etatp'] = df_2011['etatp'].fillna(-1).astype(int)
df_2012['etatp'] = df_2012['etatp'].fillna(-1).astype(int)
df_2013['etatp'] = df_2013['etatp'].fillna(-1).astype(int)
df_2014['etatp'] = df_2014['etatp'].fillna(-1).astype(int)
df_2015['etatp'] = df_2015['etatp'].fillna(-1).astype(int)
df_2016['etatp'] = df_2016['etatp'].fillna(-1).astype(int)
df_2017['etatp'] = df_2017['etatp'].fillna(-1).astype(int)
df_2018['etatp'] = df_2018['etatp'].fillna(-1).astype(int)
# Inutile pour les autres df >=2019 mais maintien à minima pour forcer le type en 'str'
df_2019['etatp'] = df_2019['etatp'].fillna(-1).astype(int)
df_2020['etatp'] = df_2020['etatp'].fillna(-1).astype(int)
df_2021['etatp'] = df_2021['etatp'].fillna(-1).astype(int)
df_2022['etatp'] = df_2022['etatp'].fillna(-1).astype(int)


In [172]:
# 10.2 Variable 'etatp' ne concerne que les piéton : il manque des infos (-1) pour des non piétons --> 0=non concerné
df_2005.loc[(df_2005['catu'] != 3) & (df_2005['etatp'] == -1), 'etatp'] = 0
df_2006.loc[(df_2006['catu'] != 3) & (df_2006['etatp'] == -1), 'etatp'] = 0
df_2007.loc[(df_2007['catu'] != 3) & (df_2007['etatp'] == -1), 'etatp'] = 0
df_2008.loc[(df_2008['catu'] != 3) & (df_2008['etatp'] == -1), 'etatp'] = 0
df_2009.loc[(df_2009['catu'] != 3) & (df_2009['etatp'] == -1), 'etatp'] = 0
df_2010.loc[(df_2010['catu'] != 3) & (df_2010['etatp'] == -1), 'etatp'] = 0
df_2011.loc[(df_2011['catu'] != 3) & (df_2011['etatp'] == -1), 'etatp'] = 0
df_2012.loc[(df_2012['catu'] != 3) & (df_2012['etatp'] == -1), 'etatp'] = 0
df_2013.loc[(df_2013['catu'] != 3) & (df_2013['etatp'] == -1), 'etatp'] = 0
df_2014.loc[(df_2014['catu'] != 3) & (df_2014['etatp'] == -1), 'etatp'] = 0
df_2015.loc[(df_2015['catu'] != 3) & (df_2015['etatp'] == -1), 'etatp'] = 0
df_2016.loc[(df_2016['catu'] != 3) & (df_2016['etatp'] == -1), 'etatp'] = 0
df_2017.loc[(df_2017['catu'] != 3) & (df_2017['etatp'] == -1), 'etatp'] = 0
df_2018.loc[(df_2018['catu'] != 3) & (df_2018['etatp'] == -1), 'etatp'] = 0
df_2019.loc[(df_2019['catu'] != 3) & (df_2019['etatp'] == -1), 'etatp'] = 0
df_2020.loc[(df_2020['catu'] != 3) & (df_2020['etatp'] == -1), 'etatp'] = 0
df_2021.loc[(df_2021['catu'] != 3) & (df_2021['etatp'] == -1), 'etatp'] = 0
df_2022.loc[(df_2022['catu'] != 3) & (df_2022['etatp'] == -1), 'etatp'] = 0


In [173]:
# 11. Réindexer les colonnes de chaque df dans le même ordre
ordre_colonnes = ['id_accident', 'id_vehicule', 'num_veh', 'id_usager', 'place', 
                  'catu', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 
                  'locp', 'actp', 'etatp', 'gravite']
df_2005 = df_2005.reindex(columns=ordre_colonnes)
df_2006 = df_2006.reindex(columns=ordre_colonnes)
df_2007 = df_2007.reindex(columns=ordre_colonnes)
df_2008 = df_2008.reindex(columns=ordre_colonnes)
df_2009 = df_2009.reindex(columns=ordre_colonnes)
df_2010 = df_2010.reindex(columns=ordre_colonnes)
df_2011 = df_2011.reindex(columns=ordre_colonnes)
df_2012 = df_2012.reindex(columns=ordre_colonnes)
df_2013 = df_2013.reindex(columns=ordre_colonnes)
df_2014 = df_2014.reindex(columns=ordre_colonnes)
df_2015 = df_2015.reindex(columns=ordre_colonnes)
df_2016 = df_2016.reindex(columns=ordre_colonnes)
df_2017 = df_2017.reindex(columns=ordre_colonnes)
df_2018 = df_2018.reindex(columns=ordre_colonnes)
df_2019 = df_2019.reindex(columns=ordre_colonnes)
df_2020 = df_2020.reindex(columns=ordre_colonnes)
df_2021 = df_2021.reindex(columns=ordre_colonnes)
df_2022 = df_2022.reindex(columns=ordre_colonnes)


In [174]:
# 12. Concaténer les df en un seul
user = df_2005.copy()
user = pd.concat ([user, df_2006], axis = 0)
user = pd.concat ([user, df_2007], axis = 0)
user = pd.concat ([user, df_2008], axis = 0)
user = pd.concat ([user, df_2009], axis = 0)
user = pd.concat ([user, df_2010], axis = 0)
user = pd.concat ([user, df_2011], axis = 0)
user = pd.concat ([user, df_2012], axis = 0)
user = pd.concat ([user, df_2013], axis = 0)
user = pd.concat ([user, df_2014], axis = 0)
user = pd.concat ([user, df_2015], axis = 0)
user = pd.concat ([user, df_2016], axis = 0)
user = pd.concat ([user, df_2017], axis = 0)
user = pd.concat ([user, df_2018], axis = 0)
user = pd.concat ([user, df_2019], axis = 0)
user = pd.concat ([user, df_2020], axis = 0)
user = pd.concat ([user, df_2021], axis = 0)
user = pd.concat ([user, df_2022], axis = 0)


In [175]:
# 14. Suppression des objets inutiles --> libération de mémoire
del df_2005
del df_2006
del df_2007
del df_2008
del df_2009
del df_2010
del df_2011
del df_2012
del df_2013
del df_2014
del df_2015
del df_2016
del df_2017
del df_2018
del df_2019
del df_2020
del df_2021
del df_2022

In [176]:
# 15. Controles et ajustements
user.info()
#user.occutc.isna().sum()

user[(user['id_accident'].astype(str).str[:4].astype(int)==2022)].head()

#user['senc'].value_counts().sort_index()
#user.head(50)
#user.catv.value_counts()
#user.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 2636377 entries, 0 to 126661
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id_accident  int64  
 1   id_vehicule  int64  
 2   num_veh      object 
 3   id_usager    object 
 4   place        int64  
 5   catu         int64  
 6   sexe         int64  
 7   an_nais      float64
 8   trajet       int64  
 9   secu1        int64  
 10  secu2        int64  
 11  secu3        int64  
 12  locp         int64  
 13  actp         object 
 14  etatp        int64  
 15  gravite      int64  
dtypes: float64(1), int64(12), object(3)
memory usage: 341.9+ MB


,id_accident,id_vehicule,num_veh,id_usager,place,catu,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp,gravite
0,202200000001,813952,A01,1 099 700,1,1,1,2008.0,5,2,8,-1,0,0,0,2
1,202200000001,813953,B01,1 099 701,1,1,1,1948.0,5,1,8,-1,0,0,0,0
2,202200000002,813950,B01,1 099 698,1,1,1,1988.0,9,1,0,-1,0,0,0,1
3,202200000002,813951,A01,1 099 699,1,1,1,1970.0,4,1,0,-1,0,0,0,0
4,202200000003,813948,A01,1 099 696,1,1,1,2002.0,-1,1,0,-1,0,0,0,0


In [177]:
# 15. Variable 'catu' : suppression de la valeur 4 - Trotinette ou roller à partir de 2018 --> vehi_catv=99 - Autre véhicule


In [178]:
# 16. Suppression des colonnes inutiles
if 'grav' in user.columns :
    drop_columns = ['grav']
    user = user.drop(drop_columns, axis = 1)

else :
    display('>>> Colonne(s) déjà supprimée(s) du DataFrame.')


'>>> Colonne(s) déjà supprimée(s) du DataFrame.'

In [179]:

# 17. Sauvegarde du fichier en csv (sans index) 
user.to_csv(repDataConcat +"usagers.csv", index=False, encoding='utf-8')

# 17.2 Sauvegarde du fichier en csv 2022 pour pré analyse des données (sans index) 
#df_2022.to_csv(repDataConcat +"usagers_2022.csv", index=False, encoding='utf-8')